## Импорт библиотек

In [1]:
# Для построения пайплайна обучения используется lightautoml, который конфликтуает с новой версией pandas. Поэтому
!pip uninstall pandas -y
!pip install --upgrade pip > installations.txt
!pip uninstall torch -y > installations.txt # конфликтует 
!pip install torch==2.0.0 > installations.txt
!pip install pandas==1.4.3 pyarrow yellowbrick polars transformers nltk gensim lightautoml fastparquet > installations.txt

Found existing installation: pandas 2.2.0
Uninstalling pandas-2.2.0:
  Successfully uninstalled pandas-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.14 requires pandas, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.14 requires pandas, which is not installed.
stable-baselines3 2.1.0 requires pandas, which is not installed.
torchaudio 2.1.2+cpu requires torch==2.1.2, but you have torch 2.0.0 which is incompatible.
torchtext 0.16.2+cpu requires torch==2.1.2, but you have torch 2.0.0 which is incompatible.
torchvision 0.16.2+cpu requires torch==2.1.2, but you have torch 2.0.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages t

In [2]:
# for dataframe
import polars as pl
import numpy as np
import pyarrow as pa
import pandas as pd
import fastparquet

# for system
import os
import time
import sys

# for metric
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# for demention decrease
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# for common functions
from collections import OrderedDict
from collections import Counter
from kaggle_secrets import UserSecretsClient
from copy import deepcopy as copy
from typing import Tuple, List

# for monitoring of models
import wandb

# for machine learning
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.report.report_deco import ReportDeco

# my functions
sys.path.append('/kaggle/input/next-orders')
from my_functions import MarketDataProcessor

## Препроцессинг данных

____________________________________________________
Использовался ноутбук с TPU и RAM 300GB, чтобы не испытывать проблем с памятью во время операционных обработок датасета.

In [3]:
# # tmp импорт в pandas
# raw = pd.read_csv('../input/sbermarket-internship-competition/train.csv')
# sub = pd.read_csv('../input/sbermarket-internship-competition/sample_submission.csv', sep = ",")

# # Приведение столбца 'cart' к int
# raw['cart'] = raw['cart'].astype(int)

# mk_data = MarketDataProcessor(raw, sub)

# # Отбрасываем выбросы - юзеров с малым количеством товаров в истории их заказов
# filtered_raw, filtered_sub, proportion = mk_data.filter_train_data()
# print(f"Процент наблюдений, используемый для тренировки: {proportion:.2f}%")

### Polars ускоряет обработку данных
__________________________________________________
В итоге мы получаем pd датафрейм с усредненным рейтингом id во всем датасете и набор временных переменных, основанных на этой метрике (рейтинг)

In [4]:
# %%time
# mk_data.get_dummies_matrix()
# mk_data.generate_time_features()
# Train = mk_data.compile_dataset()

# Test = mk_data.compile_dataset(history_flag = 1)

### X / Y 

In [5]:
# x_cols = Train.select_dtypes(include=['int', 'int8', 'int32', 'uint32', 'uint8', 'float']).drop(columns = ['target'], axis = 1).columns.tolist()
# x_cols_pca = Train.select_dtypes(include=['int', 'int8', 'int32', 'uint32', 'uint8', 'float']).drop(columns = ['user_id', 'target'], axis = 1).columns.tolist()
# print(x_cols)
# y_cols = ['target']
# print(y_cols)

### Добавляем главные компоненты

In [6]:
# scaler = RobustScaler()

# s_Train = Train.copy()
# s_Test = Test.copy()
# s_Train[x_cols_pca] = scaler.fit_transform(s_Train[x_cols_pca])
# s_Test[x_cols_pca] = scaler.transform(s_Test[x_cols_pca])

# pca = PCA(n_components=10)

# s_Train_pca = pca.fit_transform(s_Train[x_cols])
# s_Test_pca = pca.transform(s_Test[x_cols])

# Train_pca = pd.DataFrame(s_Train_pca, columns=[f'pc{i+1}' for i in range(s_Train_pca.shape[1])])
# Test_pca = pd.DataFrame(s_Test_pca, columns=[f'pc{i+1}' for i in range(s_Train_pca.shape[1])])

# s_Train[Train_pca.columns.to_list()] = Train_pca
# s_Test[Test_pca.columns.to_list()] = Test_pca

### Кластеризация id(юзер-категория)

In [7]:
# from sklearn.cluster import KMeans
# from yellowbrick.cluster.elbow import kelbow_visualizer

# print('Elbow Method to determine the number of clusters for id and target:')
# kelbow_visualizer(KMeans(random_state=4), 
#                   Train_pca, 
#                   k=(2,21))


In [8]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=5)
# cluster = kmeans.fit_predict(Train_pca)
# test_cluster = kmeans.predict(Test_pca)
# s_Train["Clusters"] = cluster
# s_Test["Clusters"] = test_cluster

### Сохранение данных в parquet

In [9]:
# Train.to_parquet('Train.parquet', index=False)
# s_Train.to_parquet('s_Train.parquet', index=False)
# s_Test.to_parquet('s_Test.parquet', index=False)

In [10]:
s_Train = pd.read_parquet('/kaggle/input/next-orders/s_Train (1).parquet')
# s_Test = pd.read_parquet('/kaggle/input/next-orders/s_Test (1).parquet')

## LightAutoML пайплайн


In [11]:
def f1 (real, pred, **kwargs):
    return f1_score(real, (pred > 0.2).astype(int), **kwargs)

N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TARGET_NAME = 'target'
ROLES = {'target': TARGET_NAME, 'drop': ['id', 'user_id', 'category'], 'category': ['Clusters']}
TASK = Task('binary', metric = f1)

In [12]:
Train_set, Valid_set = train_test_split(s_Train, test_size = TEST_SIZE,
                                        stratify = s_Train[TARGET_NAME], random_state = 23)
Train_set.reset_index(drop=True, inplace=True)
Valid_set.reset_index(drop=True, inplace=True)


In [13]:
del s_Train

In [14]:
real_estimators = ['total_order_in_cat_max',
 'ordered_mean',
 'pc2',
 'total_rating_cat_max',
 'rating_max',
 'rating_mean',
 'pc5',
 'weekday_mean',
 'pc6',
 'pc4',
 'pc10',
 'pc3',
 'pc1',
 'pc7',
 'pc8',
 'pc9',
 'rating_per_month_mean',
 'month_std',
 'month_mean',
 'rating_per_month_std',
 'weekday_std',
 'month_max',
 'ordered_std',
 'rating_std',
 'weekday_min',
 'month_min',
 't_rating_per_month_mean',
 'Clusters',
 'weekday_max',
 't_rating_per_month_sum',
 'tc_rating_per_month_mean',
 't_rating_per_month_median',
 't_rating_per_month_std',
 'tc_rating_per_month_sum',
 'ordered_max',
 'tc_rating_per_month_std']

In [15]:
Train_set[real_estimators + ['target']]

,total_order_in_cat_max,ordered_mean,pc2,total_rating_cat_max,rating_max,rating_mean,pc5,weekday_mean,pc6,pc4,...,Clusters,weekday_max,t_rating_per_month_sum,tc_rating_per_month_mean,t_rating_per_month_median,t_rating_per_month_std,tc_rating_per_month_sum,ordered_max,tc_rating_per_month_std,target
0,0.019940,0.0,-6.889078,0.017995,0.0,0.0,-1.640512,-0.925466,-0.422434,-0.597641,...,4,0.0,1.250000e-01,1.250000e-01,0.125,1.250000e-01,1.250000e-01,0.0,1.250000e-01,0
1,0.632104,0.0,-12.856729,0.275386,0.0,0.0,-1.113097,0.645963,1.158294,3.769233,...,3,0.0,-2.500000e-01,-2.500000e-01,-0.250,-2.500000e-01,-2.500000e-01,0.0,-2.500000e-01,0
2,12.365902,0.0,4.616176,5.948104,0.0,0.0,3.105760,0.360248,1.232610,2.886364,...,3,-1.0,-2.500000e-01,-2.500000e-01,-0.250,-2.500000e-01,-2.500000e-01,0.0,-2.500000e-01,0
3,0.047856,0.0,-6.050937,-0.007069,0.0,0.0,0.967077,-0.782609,-1.139021,1.133094,...,4,-2.0,-2.500000e-01,-2.500000e-01,-0.250,-2.500000e-01,-2.500000e-01,0.0,-2.500000e-01,0
4,1.052841,0.0,-13.254814,0.658098,0.0,0.0,-0.765891,-0.354037,0.215610,3.762774,...,2,0.0,-1.250000e-01,-1.250000e-01,-0.125,-1.250000e-01,-1.250000e-01,0.0,-1.250000e-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14095995,0.019940,0.0,-5.122734,-0.007712,0.0,0.0,1.303213,-1.854037,-3.402771,0.234976,...,4,-4.0,-1.250000e-01,-1.250000e-01,-0.125,-1.250000e-01,-1.250000e-01,0.0,-1.250000e-01,0
14095996,-0.055833,0.0,-19.395690,-0.054627,0.0,0.0,-0.937692,0.196983,0.282705,3.841568,...,0,0.0,2.500000e-01,2.500000e-01,0.250,2.500000e-01,2.500000e-01,0.0,2.500000e-01,0
14095997,0.060818,0.0,-19.966819,0.038078,0.0,0.0,-0.621160,-0.782609,-0.832282,5.325156,...,0,-1.0,-1.200078e-15,-3.104845e-16,0.000,-1.462479e-16,-4.229074e-16,0.0,-2.061510e-16,0
14095998,-0.048853,0.0,-4.699529,-0.055591,0.0,0.0,2.035184,-1.211180,-1.802539,1.282033,...,4,-3.0,-3.750000e-01,-3.750000e-01,-0.375,-3.750000e-01,-3.750000e-01,0.0,-3.750000e-01,0


In [16]:
%%time 

automl = TabularAutoML(
    task = TASK,
    timeout = 2400,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
    general_params = {'use_algos': [['lgb_tuned', 'cb_tuned']]}
)


train_pred = automl.fit_predict(Train_set[real_estimators + ['target']], roles = ROLES, verbose = 2)

print('Score', "%.5f" % f1(Train_set.target, train_pred.data))
valid_pred = automl.predict(Valid_set[real_estimators + ['target']])
print('Score on out of folds validation', "%.5f" % f1(Valid_set.target, valid_pred.data))

[07:42:55] Stdout logging level is INFO2.
[07:42:55] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[07:42:55] Task: binary

[07:42:55] Start automl preset with listed constraints:
[07:42:55] - time: 2400.00 seconds
[07:42:55] - CPU: 4 cores
[07:42:55] - memory: 16 GB

[07:42:55] Train data shape: (14096000, 37)



/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:37: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


[07:43:41] Layer 1 train process start. Time left 2353.81 secs
[07:59:26] Selector_LightGBM fitting and predicting completed
[08:01:55] Start hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM ... Time budget is 1.00 secs
[08:44:13] Hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM completed
[08:44:13] The set of hyperparameters {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}
 achieve 0.4312 AutoML Metric
[08:44:13] Start fitting Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM ...
[08:44:17] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM =====
[09:26:14] Time limit exceeded after calculating fold 0

[09:26:16] Fitting Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM finished. score = 0.431177159533821
[09:26:16] Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM fitting and predicting completed
[09:26:16

In [17]:
# print(automl.create_model_str_desc())

In [18]:
# fast_fi = automl.get_feature_scores('fast', silent=False)
# fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

In [19]:
import joblib

joblib.dump(automl, 'automl_lgbm.pkl')
# automl=joblib.load('/kaggle/working/automl_03.pkl')

['automl_lgbm.pkl']

In [20]:
valid_pred = automl.predict(Valid_set)
print('Score on out of folds validation', "%.5f" % f1(Valid_set.target, valid_pred.data))

Score on out of folds validation 0.42668


### Нахождение лучшего порогового значения для сепарации 0 и 1

In [21]:
# best_score = 0
# for i in np.arange(0.01, 1.0, 0.01):
#     score = f1 = f1_score(Valid_set.target, (valid_pred.data > i).astype(int))
#     if score > best_score:
#         best_score = score
#         proba_split = i

# print('At i =', "%.2f" % proba_split,'score is : ' "%.5f" % best_score)

In [22]:
# s_Test['target'] = (predictions.data > proba_split).astype(int)
# submit = pd.merge(sub['id'], s_Test[['id', 'target']], on='id')

## Подготовка сабмита

In [23]:
# import csv

# with open('submission02.csv', 'w', newline='') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerow(submit.columns)
#     for row in submit.values:
#         csvwriter.writerow(row)

##  Dense

In [24]:
# default_lama_params = {
#     "task": task,
#     "timeout": TIMEOUT,
#     "cpu_limit": N_THREADS,
#     "reader_params": {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
# }

# default_nn_params = {
#     "bs": 512, "num_workers": 4, "path_to_save": None, "n_epochs": 10, "freeze_defaults": True
# }

In [25]:
# import torch.nn as nn
# from lightautoml.automl.presets.tabular_presets import TabularAutoML

# Train_set, Valid_set = train_test_split(s_Train, test_size = TEST_SIZE,
#                                         stratify = s_Train[TARGET_NAME], random_state = 23)
# Train_set.reset_index(drop=True, inplace=True)
# Valid_set.reset_index(drop=True, inplace=True)

# automl = TabularAutoML(
#     **default_lama_params,
#     general_params={"use_algos": [["denselight_tuned"]]},
#     nn_params={
#         **default_nn_params,
#         "n_epochs": 5,
#         "tuning_params": {
#             "max_tuning_iter": 5,
#             "max_tuning_time": 500,
#             "fit_on_holdout": True
#         }
#     },
# )

# train_pred = automl.fit_predict(Train_set[real_estimators + ['target']], roles = ROLES, verbose = 2)

# print('Score', "%.5f" % f1(Train_set.target, train_pred.data))
# valid_pred = automl.predict(Valid_set[real_estimators + ['target']])
# print('Score on out of folds validation', "%.5f" % f1(Valid_set.target, valid_pred.data))